In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split
from sklearn.metrics import average_precision_score
import numpy as np
import numexpr as ne

In [2]:
train = pd.read_csv("train.csv", index_col="row_id")

xy = train.iloc[:,:2].values # units are kilometers
accuracy = train.iloc[:,2].values * 0.001 # assume accuracy is reported in meters so convert to kilometers
time = train.iloc[:,3].values # units are minutes
time_of_day = train.iloc[:,3].values % 1440 # minutes
time_of_week = train.iloc[:,3].values % 10080 # minutes
place_id = train.iloc[:,4].values

In [3]:
neigh = NearestNeighbors(n_jobs=-1, algorithm='kd_tree')
%time neigh.fit(xy, place_id)

CPU times: user 3min 20s, sys: 3.55 s, total: 3min 23s
Wall time: 3min 32s


NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [4]:
permutation = np.random.permutation(len(xy))

In [5]:
limit = 400000

validation_indicies = permutation[slice(0,min(limit,len(xy)))]

In [39]:
# from sklearn.base import BaseEstimator

# class FacebookCheckins(BaseEstimator):
    
#     def __init__(self, X_all, y_all, kNN=400, a_scale=1, a_min=1, x=0, day_w1=0, day_w2=720, day_min=0, \
#                  week_w1=0, week_w2=5040, week_min=0, time_w1=201600, time_w2=504000, time_min=1, n_jobs=-1):
#         self.X = X
#         self.y = y
#         self.place_id = place_id
#         self.kNN = kNN
#         self.a_scale = a_scale
#         self.a_min = a_min
#         self.x = x
#         self.day_w1 = day_w1
#         self.day_w2 = day_w2
#         self.day_min = day_min
#         self.week_w1 = week_w1
#         self.week_w2 = week_w2
#         self.week_min = week_min
#         self.time_w1 = time_w1
#         self.time_w2 = time_w2
#         self.time_min = time_min
#         self.n_jobs = n_jobs
        
#         self.nearest_neighbors_clf = NearestNeighbors(n_jobs=self.n_jobs, algorithm='kd_tree')
        
#         self.n_neighbors = None
        
#         self.distances = None
#         self.neighbors = None
        
#     def fit(self, X, y):
#         if not self.nearest_neighbors_clf:
#             print("creating NN classifier")
#         if self.X != X or self.y != y:
#             print("fitting NN classifier")
#             self.X = X
#             self.Y = y
#             self.nearest_neighbors_clf.fit(X[:2], y)
#         if self.distances = None or self.neighbors = None or self.n_neighbors != self.kNN:
#             self.distances, self.neighbors = self.nearest_neighbors_clf.kneighbors(X)

day = 1440
hour = 60
week = 10080

# parameters
parameters = { 'kNN': 400, 
               'accuracy' : { 'scale': 1, # 1 assumes unit is meters
                              'min': 1,
                              'bias': 0
                            },
               'time': { 
                         'day': { 'window_max': 0*hour, 'window_submax': 12*hour, 'min_prob': 0 },
                         'week': { 'window_max': 0*day, 'window_submax': 3.5*day, 'min_prob': 0 },
                         'abs': { 'window_max': 20*week, 'window_submax': 50*week, 'min_prob': 1 }
                       },
             }

def time_difference(time1, time2, period=None):
    """Find the different in time even if measure is periodic."""
    if period:
        hp = 0.5 * period
        return ne.evaluate('hp-abs(abs(time1-time2) - hp)')
    else:
        return ne.evaluate('abs(time1-time2)')

def prob_overlap_time(diff, params):
    """Compute the probability the the time difference is significant."""
    w1 = params['window_max']
    w2 = params['window_submax']
    mp = params['min_prob']
    # derive equation of line that connects end of w1 and w2
    # points: (w1, 1), (w2, mp)
    # dy = mp-1, dx = w2-w1, m = (mp-1)/(w2-w1)
    # y = m * x + b
    # substitude in point 1
    # 1 = (mp-1)/(w2-w1) * w1 + b
    # solve for b
    # b = 1 - (mp-1)/(w2-w1) * w1
    # y = (mp-1)/(w2-w1) * x + 1 - (mp-1)/(w2-w1)
    prob = ne.evaluate('(mp-1)/(w2-w1) * diff + 1 - (mp-1)/(w2-w1) * w1')
    prob = np.where(diff < w1, 1, prob)
    return np.where(diff > w2, mp, prob)

def uniqify(seq):
    """Removes duplicates from sequence and maintains order."""
    seen = set()
    seen_add = seen.add
    return np.fromiter((x for x in seq if not (x in seen or seen_add(x))), dtype=np.int64)

def prob_overlap_locations(x1, y1, x2, y2, accuracy1, accuracy2):
    """Compute the probability that location measurements represent the same point."""
#     dist2 = np.sum(ne.evaluate('(xy1-xy2)**2'), axis=-1)
    return ne.evaluate('exp(-0.5 * ((x1-x2)**2+(y1-y2)**2) / (accuracy1 ** 2 + accuracy2 ** 2)) / \
                        (accuracy1 ** 2 + accuracy2 ** 2)') # / (2 * np.pi)

def sum_by_group(values, groups):
    """Sum a list of values by groups."""
    order = np.argsort(groups)
    groups = groups[order]
    values = values[order]
    values.cumsum(out=values)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values = values[index]
    groups = groups[index]
    values[1:] = values[1:] - values[:-1]
    return values, groups

def scale_accuracy(accuracy, params):
    scale = params['scale']
    bias = params['bias']
    a_min = params['min']
    return np.maximum(accuracy + bias, a_min) * scale
    
def predict_xy_accuracy_time(test_points, neighbors, parameters, self_validation=False):
    
    neighbor_accuracies = scale_accuracy(accuracy[neighbors], parameters['accuracy'])
    test_accuracy = scale_accuracy(accuracy[test_points, None], parameters['accuracy'])
    colocation_prob = prob_overlap_locations(xy[test_points, 0, None], xy[test_points, 1, None], \
                                             xy[neighbors, 0], xy[neighbors, 1], test_accuracy, neighbor_accuracies)
    
    time_of_day_diff = time_difference(time_of_day[test_points, None], time_of_day[neighbors], day)
    time_of_day_prob = prob_overlap_time(time_of_day_diff, parameters['time']['day'])
    
    time_of_week_diff = time_difference(time_of_week[test_points, None], time_of_week[neighbors], week)
    time_of_week_prob = prob_overlap_time(time_of_week_diff, parameters['time']['week'])
    
    time_abs_diff = time_difference(time[test_points, None], time[neighbors])
    time_abs_prob = prob_overlap_time(time_abs_diff, parameters['time']['abs'])
    
    total_prob = ne.evaluate('colocation_prob * time_of_day_prob * time_of_week_prob * time_abs_prob')
    
    s = slice(1,None) if self_validation else slice(0,None) # skip the first neighbor if self validating
    predictions = np.zeros((len(distances),3))
    for i, (prob, places) in enumerate(zip(total_prob[:,s], place_id[neighbors][:,s])):
        # append a few zeros just incase there is only one nearby place
        # we need three for the precision calculation
        prob, places = sum_by_group(np.append(prob, [0,0]), np.append(places, [0,1]))
        prob, places = zip(*sorted(zip(prob, places),reverse=True))
        predictions[i,:] = places[:3]
    return predictions
        
def mean_average_precision3(true, test):
    precision = np.array([1, 1/2, 1/3])
    return ne.evaluate('sum((true == test) * precision)') / len(true)

In [7]:
print("find nearest neighbors")
%time neighbors = neigh.kneighbors(xy[validation_indicies], n_neighbors=parameters['kNN'], \
                                   return_distance=False).astype(np.int32)

# del neigh # free up lots of memory

find nearest neighbors
CPU times: user 1min 39s, sys: 21.6 s, total: 2min 1s
Wall time: 48.7 s


In [40]:
print("predict")
%time predictions = predict_xy_accuracy_time(validation_indicies, neighbors, parameters, \
                                             self_validation=True)

print("evaluate")
%time mean_average_precision3(place_id[validation_indicies, None], predictions)

predict
CPU times: user 1min 50s, sys: 2min 10s, total: 4min 1s
Wall time: 3min 59s
evaluate
CPU times: user 64.9 ms, sys: 408 ms, total: 473 ms
Wall time: 541 ms


0.57425750000021858

In [44]:
from sklearn.grid_search import ParameterSampler

params = {'a': eval('[1,2,3]')}

list(ParameterSampler(params, n_iter=3))

[{'a': 1}, {'a': 2}, {'a': 3}]